In [ ]:
import os
import re
import requests

import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [ ]:
#FILE_PATH = os.path.dirname(os.path.dirname(os.path.abspath(__file__))) #Usado em arquivo .py
FILE_PATH = os.path.join(os.path.abspath('.')) # Usado em arquivo .ipynb (Contém o caminho deste arquivo)
SESSION = requests.session()
BASE_URL = 'https://harrypotter.fandom.com'

In [ ]:
# Lista de personagens (codes) considerados para o network
characters_urls = ['/wiki/Harry_Potter',
                    '/wiki/Dudley_Dursley',
                    '/wiki/Vernon_Dursley',
                    '/wiki/Arthur_Weasley',
                    '/wiki/Molly_Weasley',
                    '/wiki/William_Weasley',
                    '/wiki/Fleur_Delacour',
                    '/wiki/Victoire_Weasley',
                    '/wiki/Dominique_Weasley',
                    '/wiki/Louis_Weasley',
                    '/wiki/Charles_Weasley',
                    '/wiki/Percy_Weasley',
                    '/wiki/Audrey_Weasley',
                    '/wiki/Molly_Weasley_II',
                    '/wiki/Lucy_Weasley',
                    '/wiki/Fred_Weasley',
                    '/wiki/George_Weasley',
                    '/wiki/Angelina_Johnson',
                    '/wiki/Fred_Weasley_II',
                    '/wiki/Roxanne_Weasley',
                    '/wiki/Ronald_Weasley',
                    '/wiki/Hermione_Granger',
                  '/wiki/Rose_Granger-Weasley',
                  '/wiki/Hugo_Granger-Weasley']

In [ ]:
# Processo de iteração

# Lista que representam os futuros datasets
relationships = []
characters_unique = []
for character in tqdm(characters_urls, desc='CRAWLING CHARACTERS DATA'):
    # União de strings da url básico com a termição da url de cada personagem
    char_page = SESSION.get(BASE_URL+character)
    bs = BeautifulSoup(char_page.content, 'html.parser')
    
    character = re.sub(r'(\_|\/wiki\/)', ' ', character).strip() # Contém o nome do personagem
    image_character = bs.find(class_='pi-image-thumbnail')['src'] # Contém a url da imagem que representa o personagem na página
    characters_unique.append({'character': character, 'characters_unique_url': image_character}) # Preenchimento do primeiro dataset
    
    char_family_info = bs.find(attrs={"data-source": "family"}).find_all(class_='pi-data-value pi-font')
    
    # Processo de obtenção dos familiares de cada personagem
    for idx, relative in enumerate(zip(char_family_info[0].find_all('li'))):
        relative_data = {}
        
        relationship = re.findall('\((.*?)\)', relative[0].text) # O tipo de relacionamento do personagem com cada familiar foi escrita no site entre ()
        if len(relationship) == 1:
            relationship = relationship[0]
        if len(relationship) == 2:
            relationship = relationship[1]
        
        relative_data['character'] = character # Contém o nome do personagem
        relative_data['relative'] = relative[0].text.split(' (')[0] # Contém o nome do familiar
        relative_data['relationship'] = relationship # Contém o tipo de relacionamento de parentesco entre o personagem e o familiar
        
        relationships.append(relative_data) ## Preenchimento do primeiro dataset

In [ ]:
# Transformação das listas em dataframes
characters_dataset = pd.DataFrame(relationships)
characters_unique_dataset = pd.DataFrame(characters_unique)

# Processamento dos arquivos .csv
characters_dataset.to_csv(FILE_PATH + '/data/raw/characters_dataset.csv', index=False)
characters_unique_dataset.to_csv(FILE_PATH + '/data/raw/characters_unique_dataset.csv', index=False)